- Make a chain that takes the name of a movie and replies with information about the movie such as the director, the main cast, the budget, the box office revenue, the genre of the movie and a small synopsis.  
  영화 이름을 가지고 감독, 주요 출연진, 예산, 흥행 수익, 영화의 장르, 간단한 시놉시스 등 영화에 대한 정보로 답장하는 체인을 만드세요.
- The LLM should always reply using the same format, to accomplish this you must give examples to the LLM of your desired output.  
  LLM은 항상 동일한 형식을 사용하여 응답해야 하며, 이를 위해서는 원하는 출력의 예시를 LLM에 제공해야 합니다.
- To provide examples use FewShotPromptTemplate or FewShotChatMessagePromptTemplate.  
  예제를 제공하려면 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하세요.


In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector.base import BaseExampleSelector


chat = ChatOpenAI(
    temperature=0.2,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# FewShotPromptTemplate


In [2]:
examples = [
    {
        "question": "Tell me about the movie Inception.",
        "answer": """
        Here is the information about the movie Inception:

        Director: Christopher Nolan  
        Main Cast: Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page, Tom Hardy  
        Budget: $160 million  
        Box Office Revenue: $836.8 million  
        Genre: Science Fiction, Thriller, Action  
        Synopsis: A skilled thief, Dom Cobb, who steals secrets through dream invasion, is offered a chance to erase his criminal history if he can plant an idea in someone's mind using a process called 'Inception.'  
        """,
    },
    {
        "question": "What can you tell me about Parasite?",
        "answer": """
        Here is the information about the movie Parasite:

        Director: Bong Joon-ho  
        Main Cast: Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong, Choi Woo-shik, Park So-dam  
        Budget: $11 million  
        Box Office Revenue: $263 million  
        Genre: Thriller, Drama, Dark Comedy  
        Synopsis: The story follows the Kim family, who scheme to become employed by a wealthy family, the Parks, by infiltrating their household through deception, leading to unexpected and tragic consequences.  
        """,
    },
]

In [3]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate


class RandomExampleSeletor(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        print("input_variables:", input_variables)
        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = RandomExampleSeletor(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: Tell me about the movie {movie}",
    input_variables=["movie"],
)

prompt.format(movie="Harry Potter and the Sorcerer's Stone")

input_variables: {'movie': "Harry Potter and the Sorcerer's Stone"}


"Human: What can you tell me about Parasite?\nAI: \n        Here is the information about the movie Parasite:\n\n        Director: Bong Joon-ho  \n        Main Cast: Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong, Choi Woo-shik, Park So-dam  \n        Budget: $11 million  \n        Box Office Revenue: $263 million  \n        Genre: Thriller, Drama, Dark Comedy  \n        Synopsis: The story follows the Kim family, who scheme to become employed by a wealthy family, the Parks, by infiltrating their household through deception, leading to unexpected and tragic consequences.  \n        \n\nHuman: Tell me about the movie Harry Potter and the Sorcerer's Stone"

In [4]:
chain = prompt | chat
chain.invoke({"movie": "Harry Potter and the Sorcerer's Stone"})

input_variables: {'movie': "Harry Potter and the Sorcerer's Stone"}
AI: 
        Here is the information about the movie Harry Potter and the Sorcerer's Stone:

        Director: Chris Columbus  
        Main Cast: Daniel Radcliffe, Rupert Grint, Emma Watson, Robbie Coltrane, Richard Harris  
        Budget: $125 million  
        Box Office Revenue: $974.8 million  
        Genre: Fantasy, Adventure  
        Synopsis: The film follows the story of a young boy named Harry Potter who discovers he is a wizard and attends Hogwarts School of Witchcraft and Wizardry, where he learns about his past and his destiny to defeat the dark wizard Lord Voldemort.

AIMessageChunk(content="AI: \n        Here is the information about the movie Harry Potter and the Sorcerer's Stone:\n\n        Director: Chris Columbus  \n        Main Cast: Daniel Radcliffe, Rupert Grint, Emma Watson, Robbie Coltrane, Richard Harris  \n        Budget: $125 million  \n        Box Office Revenue: $974.8 million  \n        Genre: Fantasy, Adventure  \n        Synopsis: The film follows the story of a young boy named Harry Potter who discovers he is a wizard and attends Hogwarts School of Witchcraft and Wizardry, where he learns about his past and his destiny to defeat the dark wizard Lord Voldemort.")

# Load Prompt


In [5]:
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.json")
prompt.format(movie="Wicked")

# load_prompt는 파일의 최상위 레벨에서 _type 키를 사용해 어떤 종류의 프롬프트를 로드할지 결정한다
# "_type" 키가 없으면 : No `_type` key found, defaulting to `prompt`.
# 키 이름은 정해져 있는 듯 (e.g "template" -> KeyError: 'example_prompt')

"Human: Tell me about the movie Inception.\nAI: Director: Christopher Nolan\nMain Cast: Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page, Tom Hardy\nBudget: $160 million\nBox Office Revenue: $836.8 million\nGenre: Science Fiction, Thriller, Action\nSynopsis: A skilled thief, Dom Cobb, who steals secrets through dream invasion, is offered a chance to erase his criminal history if he can plant an idea in someone's mind using a process called 'Inception.'\n\nHuman: What can you tell me about Parasite?\nAI: Director: Bong Joon-ho\nMain Cast: Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong, Choi Woo-shik, Park So-dam\nBudget: $11 million\nBox Office Revenue: $263 million\nGenre: Thriller, Drama, Dark Comedy\nSynopsis: The story follows the Kim family, who scheme to become employed by a wealthy family, the Parks, by infiltrating their household through deception, leading to unexpected and tragic consequences.\n\nHuman: Tell me about the movie Wicked"

In [6]:
# "suffix": "Human: Tell me about the movie {movie}\nAI: Please respond with the same format as the examples above."
chain = prompt | chat
response = chain.invoke({"movie": "Wicked"})

print(response.content)  # 개봉한지 얼마 안돼서 반영 못하는 것으로 보임

AI: "Wicked" is a musical fantasy film based on the Broadway musical of the same name, which is a retelling of "The Wizard of Oz" from the perspective of the witches of Oz, Glinda and Elphaba. The film adaptation is currently in development, with Stephen Daldry set to direct. The release date and casting details have not been announced yet.AI: "Wicked" is a musical fantasy film based on the Broadway musical of the same name, which is a retelling of "The Wizard of Oz" from the perspective of the witches of Oz, Glinda and Elphaba. The film adaptation is currently in development, with Stephen Daldry set to direct. The release date and casting details have not been announced yet.


In [7]:
# "suffix": "Human: Tell me about the movie {movie}"
response = chain.invoke({"movie": "Iron Man"})

print(response.content)

AI: Director: Jon Favreau
Main Cast: Robert Downey Jr., Gwyneth Paltrow, Jeff Bridges, Terrence Howard
Budget: $140 million
Box Office Revenue: $585.2 million
Genre: Action, Sci-Fi
Synopsis: The film follows Tony Stark, a billionaire industrialist and genius inventor, who is kidnapped and forced to build a devastating weapon. Instead, he creates a suit of armor to escape and becomes the superhero Iron Man, using his technology to fight against evil.AI: Director: Jon Favreau
Main Cast: Robert Downey Jr., Gwyneth Paltrow, Jeff Bridges, Terrence Howard
Budget: $140 million
Box Office Revenue: $585.2 million
Genre: Action, Sci-Fi
Synopsis: The film follows Tony Stark, a billionaire industrialist and genius inventor, who is kidnapped and forced to build a devastating weapon. Instead, he creates a suit of armor to escape and becomes the superhero Iron Man, using his technology to fight against evil.


# FewShotChatMessagePromptTemplate


In [8]:
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate

In [12]:
example_selector = RandomExampleSeletor(examples=examples)

# example prompt
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "{answer}"),
    ]
)

# FewShotChatMessagePromptTemplate
prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
)

# final prompt
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a well-known movie youtuber"),
        prompt,
        # example_prompt,
        ("human", "What can you tell me about {question}?"),
    ]
)

final_prompt.format(question="Pirates of the Caribbean")

input_variables: {}


"System: You are a well-known movie youtuber\nHuman: Tell me about the movie Inception.\nAI: \n        Here is the information about the movie Inception:\n\n        Director: Christopher Nolan  \n        Main Cast: Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page, Tom Hardy  \n        Budget: $160 million  \n        Box Office Revenue: $836.8 million  \n        Genre: Science Fiction, Thriller, Action  \n        Synopsis: A skilled thief, Dom Cobb, who steals secrets through dream invasion, is offered a chance to erase his criminal history if he can plant an idea in someone's mind using a process called 'Inception.'  \n        \nHuman: What can you tell me about Pirates of the Caribbean?"

In [13]:
chain = final_prompt | chat
chain.invoke({"question": "Harry Potter and the Sorcerer's Stone"})

input_variables: {}

        Here is the information about the movie Harry Potter and the Sorcerer's Stone:

        Director: Chris Columbus  
        Main Cast: Daniel Radcliffe, Rupert Grint, Emma Watson, Robbie Coltrane, Maggie Smith  
        Budget: $125 million  
        Box Office Revenue: $974.8 million  
        Genre: Fantasy, Adventure  
        Synopsis: The film follows the story of a young boy named Harry Potter who discovers he is a wizard and is invited to attend Hogwarts School of Witchcraft and Wizardry. Along with his new friends Ron and Hermione, Harry uncovers the mystery of the Sorcerer's Stone and faces the dark wizard Voldemort.

AIMessageChunk(content="\n        Here is the information about the movie Harry Potter and the Sorcerer's Stone:\n\n        Director: Chris Columbus  \n        Main Cast: Daniel Radcliffe, Rupert Grint, Emma Watson, Robbie Coltrane, Maggie Smith  \n        Budget: $125 million  \n        Box Office Revenue: $974.8 million  \n        Genre: Fantasy, Adventure  \n        Synopsis: The film follows the story of a young boy named Harry Potter who discovers he is a wizard and is invited to attend Hogwarts School of Witchcraft and Wizardry. Along with his new friends Ron and Hermione, Harry uncovers the mystery of the Sorcerer's Stone and faces the dark wizard Voldemort.")